In [4]:
import numpy as np
import pandas as pd

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [7]:
df = pd.read_csv("covid_toy.csv")

In [8]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [9]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [11]:
X_train

,age,gender,fever,cough,city
23,80,Female,98.0,Mild,Delhi
47,18,Female,104.0,Mild,Bangalore
77,8,Female,101.0,Mild,Kolkata
18,64,Female,98.0,Mild,Bangalore
49,44,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
52,47,Female,100.0,Strong,Bangalore
20,12,Male,98.0,Strong,Bangalore
38,49,Female,101.0,Mild,Delhi
16,69,Female,103.0,Mild,Kolkata


# Without Column transformer


In [12]:
# imputer for handling missing values in fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also for the test
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [13]:
# Ordinalencoding-> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [14]:
# OneHotEncoding -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# for test part
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city

array([[0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 1.],


In [15]:
# extracting the age
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [16]:
# concatinating
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

# for test also the same
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed


array([[ 80.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 18.        , 104.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  8.        , 101.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 64.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 44.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 11.        , 100.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 10.        , 100.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 17.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 19.        , 101.        ,   1.        ,   1.        ,
          0.    

# With Column Transformer

In [17]:
from sklearn.compose import ColumnTransformer

In [24]:
# This column transformer has 2 parameters one for the transforme and the other for the remiander which means that
# after applying the ones which left unka kya kerna hai.
transformer = ColumnTransformer(transformers=[
    # we will pas the tuples
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'),['gender','city'])
],remainder='passthrough')

In [26]:
transformer.fit_transform(X_train)

array([[ 98.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  80.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   8.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  44.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  11.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  10.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  17.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.    